In [ ]:
#https://www.data.go.kr/data/15000563/openapi.do

In [1]:
import configparser
# ConfigParser 초기화
config = configparser.ConfigParser()
# keys.config 파일 읽기
config.read('C:/Users/user/Desktop/24-2/졸업프로젝트/project_ai/keys.config')

['C:/Users/user/Desktop/24-2/졸업프로젝트/project_ai/keys.config']

In [ ]:
def extract_stage_from_address(address):
    """
    도로명 주소에서 시도(STAGE1)와 시군구(STAGE2)를 추출합니다.
    :param address: 도로명 주소 (예: "서울특별시 강남구 삼성로 123")
    :return: 시도(STAGE1), 시군구(STAGE2)
    """
    try:
        # 주소를 공백으로 분리
        parts = address.split()
        if len(parts) < 2:
            raise ValueError("주소 형식이 잘못되었습니다. 최소 시도와 시군구가 필요합니다.")
        
        # 시도와 시군구 추출
        stage1 = parts[0]  # 첫 번째는 시도
        stage2 = parts[1]  # 두 번째는 시군구

        return stage1, stage2
    except Exception as e:
        print(f"주소 파싱 오류: {e}")
        return None, None

In [ ]:
import math
import requests
import xml.etree.ElementTree as ET

# API 호출 함수
def call_api(url, params):
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        return response.text  # XML 응답
    except requests.exceptions.RequestException as e:
        print(f"API 호출 오류: {e}")
        return None


In [ ]:
def get_hospitals_by_condition(stage1, stage2, conditions):
    """
    중증질환 조건에 맞는 병원의 hpid를 필터링합니다.
    :param stage1: 시도 (STAGE1)
    :param stage2: 시군구 (STAGE2)
    :param conditions: 조건 목록 (예: ["mkioskty8", "mkioskty9"])
    :return: hpid 목록
    """
    url = "http://apis.data.go.kr/B552657/ErmctInfoInqireService/getSrsillDissAceptncPosblInfoInqire"
    params = {
        "STAGE1": stage1,
        "STAGE2": stage2,
        "pageNo": 1,
        "numOfRows": 100,
        "serviceKey": config['API_KEYS']['public_portal_api_key']  
        }
    hpid_list = []

    # 첫 호출로 totalCount 확인
    data = call_api(url, params)
    if not data:
        return []

    root = ET.fromstring(data)
    total_count = int(root.find("body/totalCount").text)
    total_pages = math.ceil(total_count / params["numOfRows"])

    # 모든 페이지 데이터 수집
    for page in range(1, total_pages + 1):
        params["pageNo"] = page
        page_data = call_api(url, params)
        if not page_data:
            continue
        page_root = ET.fromstring(page_data)
        items = page_root.findall(".//item")
        for item in items:
            if any(item.find(cond).text == "Y" for cond in conditions):  # 조건 만족
                hpid = item.find("hpid").text
                hpid_list.append(hpid)
    
    return hpid_list

In [ ]:
def get_real_time_bed_info(stage1, stage2, hpid_list):
    """
    응급실 실시간 데이터를 조회하고, 필터링된 hpid에 해당하는 데이터만 반환합니다.
    :param stage1: 시도 (STAGE1)
    :param stage2: 시군구 (STAGE2)
    :param hpid_list: 필터링된 병원의 hpid 목록
    :return: 병원 실시간 데이터 목록
    """
    url = "http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire"
    params = {
        "STAGE1": stage1,
        "STAGE2": stage2,
        "pageNo": 1,
        "numOfRows": 100,
        "serviceKey": config['API_KEYS']['public_portal_api_key'],
    }
    result = []

    # 첫 호출로 totalCount 확인
    data = call_api(url, params)
    if not data:
        return []

    root = ET.fromstring(data)
    total_count = int(root.find("body/totalCount").text)
    total_pages = math.ceil(total_count / params["numOfRows"])

    # 모든 페이지 데이터 수집
    for page in range(1, total_pages + 1):
        params["pageNo"] = page
        page_data = call_api(url, params)
        if not page_data:
            continue
        page_root = ET.fromstring(page_data)
        items = page_root.findall(".//item")
        for item in items:
            hpid = item.find("hpid").text
            if hpid in hpid_list:  # 필터링된 hpid만 포함
                result.append({
                    "hpid": hpid,
                    "phpid": item.find("phpid").text if item.find("phpid") is not None else None,
                    "hvidate": item.find("hvidate").text,
                    "hvec": item.find("hvec").text,
                    "hvoc": item.find("hvoc").text,
                    "hvcc": item.find("hvcc").text,
                    "hvncc": item.find("hvncc").text,
                    "hvccc": item.find("hvccc").text,
                    "hvicc": item.find("hvicc").text,
                    "hvgc": item.find("hvgc").text,
                    "hvdnm": item.find("hvdnm").text,
                    "hvctayn": item.find("hvctayn").text,
                    "hvmriayn": item.find("hvmriayn").text,
                    "hvangioayn": item.find("hvangioayn").text,
                    "hvventiayn": item.find("hvventiayn").text,
                    "hvamyn": item.find("hvamyn").text,
                    "hv1": item.find("hv1").text,
                    "hv2": item.find("hv2").text,
                    "hv3": item.find("hv3").text,
                    "hv4": item.find("hv4").text,
                    "hv5": item.find("hv5").text,
                    "hv6": item.find("hv6").text,
                    "hv7": item.find("hv7").text,
                    "hv8": item.find("hv8").text,
                    "hv9": item.find("hv9").text,
                    "hv10": item.find("hv10").text,
                    "hv11": item.find("hv11").text,
                    "hv12": item.find("hv12").text,
                    "dutyname": item.find("dutyName").text,
                    "dutytel3": item.find("dutytel3").text,
                })
    
    return result


In [ ]:
if __name__ == "__main__":
    # 사용자 입력
    address = "서울특별시 강남구 삼성로 123"
    conditions = ["mkioskty8", "mkioskty10"]  # 조산산모, 신생아

    # 주소에서 시도와 시군구 추출
    stage1, stage2 = extract_stage_from_address(address)

    # 1. 중증질환 조건 필터링
    hpid_list = get_hospitals_by_condition(stage1, stage2, conditions)

    # 2. 실시간 병상 정보 조회
    if hpid_list:
        real_time_data = get_real_time_bed_info(stage1, stage2, hpid_list)
        for hospital in real_time_data:
            print(hospital)
    else:
        print("조건에 맞는 병원이 없습니다.")
